In [ ]:
import ee
import pandas as pd
import time
from datetime import datetime, timedelta
import os


project_id='ee-dhanikondav'
ee.Initialize(project=project_id)
print(f"✅ Earth Engine initialized successfully with project: {project_id}")


# Define Sundarbans region (simplified bounding box)
sundarbans_geo = ee.Geometry.Rectangle([88.0, 21.5, 89.5, 22.2])

# Output directory 
output_dir = r'C:\Users\dhani\OneDrive\Desktop\satelite_image\project\csv'
os.makedirs(output_dir, exist_ok=True)

def collect_sentinel2_mangrove_data():
    """
    Collect mangrove data using Sentinel-2 imagery for selected dates
    """
    print("\n==== COLLECTING MANGROVE DATA WITH SENTINEL-2 ====")
    
    # Define factors for carbon calculations
    carbon_density = 447  # tC/ha
    sequestration_rate = 10  # tonnes CO2/ha/yr


    start_date = datetime(2024, 3, 12)
    end_date = datetime(2025, 3, 12)
    delta = timedelta(days=7)

    test_dates = []
    current_date = start_date

    while current_date <= end_date:
        test_dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += delta
    
    # Container for results
    results = []
    
    for base_date in test_dates:
        # Use a 15-day window to find imagery
        start_date = base_date
        
        # Calculate end date (15 days after start)
        base_dt = datetime.strptime(base_date, '%Y-%m-%d')
        end_dt = base_dt + timedelta(days=15)
        end_date = end_dt.strftime('%Y-%m-%d')
        
        print(f"\nProcessing period {start_date} to {end_date}...")
        
        try:
            # Get Sentinel-2 collection for this date range
            sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR") \
                .filterDate(start_date, end_date) \
                .filterBounds(sundarbans_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
                .sort('CLOUDY_PIXEL_PERCENTAGE')
            
            # Check if we have images
            collection_size = sentinel_collection.size().getInfo()
            print(f"  Found {collection_size} Sentinel-2 images")
            
            if collection_size == 0:
                print(f"  No clear Sentinel-2 images available for {base_date}")
                continue
            
            # Create a median composite to reduce cloud influence
            composite = sentinel_collection.median()
            
            # Calculate indices
            # Scale the bands (Sentinel-2 data needs to be divided by 10000)
            scaled_img = composite.divide(10000)
            
            # NDVI = (NIR - Red) / (NIR + Red) - using bands B8 and B4
            ndvi = scaled_img.normalizedDifference(['B8', 'B4']).rename('NDVI')
            
            # NDWI = (Green - NIR) / (Green + NIR) - using bands B3 and B8
            ndwi = scaled_img.normalizedDifference(['B3', 'B8']).rename('NDWI')
            
            # EVI = 2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))
            evi = scaled_img.expression(
                '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                    'NIR': scaled_img.select('B8'),
                    'RED': scaled_img.select('B4'),
                    'BLUE': scaled_img.select('B2')
                }
            ).rename('EVI')
            
            # Identify potential mangrove areas
            # Mangroves typically have high NDVI and low/negative NDWI
            mangrove_mask = ndvi.gt(0.3)  # NDVI > 0.3 for vegetation
                
            # Calculate area
            pixel_area = ee.Image.pixelArea().divide(10000)  # m² → ha
            mangrove_area_img = mangrove_mask.multiply(pixel_area)
            
            # Sum the area
            mangrove_area_sum = mangrove_area_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=sundarbans_geo,
                scale=30,
                maxPixels=1e12
            ).getInfo()
            
            # Extract the area value
            mangrove_area_ha = mangrove_area_sum.get('NDVI')
            
            if mangrove_area_ha is None:
                print(f"  Could not calculate area. Available keys: {list(mangrove_area_sum.keys())}")
                continue
            
            # Calculate carbon metrics
            carbon_stock = mangrove_area_ha * carbon_density
            co2_absorption = mangrove_area_ha * sequestration_rate
            
            # Get mean indices for mangrove areas
            indices = ee.Image.cat([ndvi, ndwi, evi]).updateMask(mangrove_mask)
            indices_stats = indices.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=sundarbans_geo,
                scale=30,
                maxPixels=1e12
            ).getInfo()
            
            # Get the values
            mean_ndvi = indices_stats.get('NDVI')
            mean_ndwi = indices_stats.get('NDWI')
            mean_evi = indices_stats.get('EVI')
            
            # Add to results
            year = int(base_date.split('-')[0])
            results.append({
                'date': base_date,
                'year': year,
                'NDVI': mean_ndvi,
                'NDWI': mean_ndwi,
                'EVI': mean_evi,
                'total_mangrove_area_ha': mangrove_area_ha,
                'total_carbon_stock_tonnes': carbon_stock,
                'total_CO2_absorption_tonnes_per_year': co2_absorption
            })
            
            print(f"  Successfully processed period around {base_date}")
            print(f"  Mangrove area: {mangrove_area_ha:.2f} ha")
            print(f"  Mean NDVI: {mean_ndvi:.4f}")
            print(f"  Mean EVI: {mean_evi:.4f}")
            
        except Exception as e:
            print(f"  Error processing {base_date}: {str(e)}")
    
    # Create DataFrame
    if results:
        df = pd.DataFrame(results)
        
        # Save to CSV
        csv_path = os.path.join(output_dir, 'mangrove_metrics_summary.csv')
        df.to_csv(csv_path, index=False)
        print(f"\nSaved mangrove data to {csv_path}")
        
        return df
    else:
        print("No data collected.")
        return None

# Run the collection function
mangrove_df = collect_sentinel2_mangrove_data()

# Display the results if available
if mangrove_df is not None:
    print("\nCollected Mangrove Data:")
    print(mangrove_df)

✅ Earth Engine initialized successfully with project: ee-dhanikondav

==== COLLECTING MANGROVE DATA WITH SENTINEL-2 ====

Processing period 2024-03-12 to 2024-03-27...


c:\Users\dhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


  Found 14 Sentinel-2 images
  Successfully processed period around 2024-03-12
  Mangrove area: 343669.48 ha
  Mean NDVI: 0.3958
  Mean EVI: -506477763.8523

Processing period 2024-03-19 to 2024-04-03...
  Found 20 Sentinel-2 images
  Successfully processed period around 2024-03-19
  Mangrove area: 250005.57 ha
  Mean NDVI: 0.3824
  Mean EVI: 0.4896

Processing period 2024-03-26 to 2024-04-10...
  Found 21 Sentinel-2 images
  Successfully processed period around 2024-03-26
  Mangrove area: 219224.03 ha
  Mean NDVI: 0.3785
  Mean EVI: 0.4893

Processing period 2024-04-02 to 2024-04-17...
  Found 21 Sentinel-2 images
  Successfully processed period around 2024-04-02
  Mangrove area: 358625.79 ha
  Mean NDVI: 0.3993
  Mean EVI: 0.4536

Processing period 2024-04-09 to 2024-04-24...
  Found 27 Sentinel-2 images
  Successfully processed period around 2024-04-09
  Mangrove area: 450647.62 ha
  Mean NDVI: 0.3990
  Mean EVI: 0.4027

Processing period 2024-04-16 to 2024-05-01...
  Found 27 Senti